In [15]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import tqdm
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os
import pickle
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)



cuda


In [2]:
# temp = pd.read_csv(os.path.join('mnt', 'source', 'temperature.csv'), delimiter=';', encoding='utf-8')
# 
temp = pd.read_csv(os.path.join('..', 'input_data', 'temperature.csv'), delimiter=';', encoding='utf-8')
with open(os.path.join('..', 'model', 'houses.pkl'), 'rb') as f:
    loaded = pickle.load(f)

houses = loaded['houses']
# houses.head()

In [3]:
new_temp = []

current_date = 0
temps=[]
humids=[]
for index, row in temp.iterrows():
    if row['date_start'].split()[0] != current_date:
        new_temp.append([current_date, np.array(temps), np.array(humids)])
        current_date = row['date_start'].split()[0]
        temps = []
        humids = []
    temps.append(float(row['temp'].replace(',', '.')))
    humids.append(row['humidity'])
    

    
# print(new_temp, len(temps))
new_temp = pd.DataFrame(new_temp, columns=['date', 'temp', 'humidity'])
temp = new_temp.drop(axis = 0, index=0)
# temp

In [4]:
model = nn.Sequential(
    nn.Linear(32, 128),
    nn.ReLU(),
    nn.Linear(128, 64), 
    nn.ReLU(),
    nn.Linear(64, 1)
)
model.load_state_dict(torch.load(os.path.join('..', 'model', 'model.pt')))

<All keys matched successfully>

In [5]:
loaded

{'houses':                              address_uuid  build_year  floor_number  \
 0    82c2ad6e-27db-4541-8e5d-00163bbe622e      1976.0           5.0   
 1    e68f3476-3f10-4374-b682-00de840dae3c      1978.0           5.0   
 2    e9eb42b5-40b1-459b-b75d-00e01b3dca51      1975.0           5.0   
 3    62f5beff-2753-490c-8115-01cfa320ee40      2002.0          10.0   
 4    90d1cab6-1aa0-4c28-9df9-01f930e6dede      1967.0           5.0   
 ..                                    ...         ...           ...   
 559  1ffbc894-9192-48ca-91c6-fd724087f909      1968.0           5.0   
 560  a166db0c-e296-45f6-ae2f-fe44ff0fc9e9      1967.0           9.0   
 561  261f583f-faf3-48f9-954c-ff912773b876      1974.0           5.0   
 562  5ebc28a8-7415-4c43-9554-ffb14cdf534c      1990.0           5.0   
 563  196eccc6-82f1-4d06-8b49-ffb152f848b3      1983.0           9.0   
 
      residential_area  roof_area_total  roof_area_metal  roof_area_web  \
 0             4621.25          1234.00          

In [19]:
result = []

print(houses.shape)
for index_h, house_row in tqdm(houses.iterrows(), total=houses.shape[0]):
    summa = 0
    
    for index_t, temp_row in temp.iterrows():
        index
        a = temp_row['temp'].tolist()[:8]
        b = temp_row['humidity'].tolist()[:8]
        c = house_row.iloc[1:].tolist()
        while (len(a)<8):
            a += [0]
        while (len(b) < 8):
            b += [0]
        
        x = ([  a + b + c]) 
        x = loaded['scaler'].transform(x)
        x = np.insert(x, 0, 0, axis=1)
        x = torch.FloatTensor(x)
        pred = model(x)
        pred = float(pred.cpu().detach()[0][0])
        summa += pred
        
    result.append([house_row['address_uuid'], summa])
#         pred = loaded['temp_scaler'].inverse_transform(pred.cpu().detach())
#         print(len(row['temp'].tolist()), len(row['humidity'].tolist()), len(house_row.iloc[1:].tolist()))
#         print(len(x[0]))
        
#             tmp = (((houses.loc[houses['address_uuid'] == row['address_uuid']]).iloc[:, 1:]))
            
#             df.append([   row['value'], row['is_unreliable']] + 
#                       (((temp.loc[temp['date'] == row['date']])['temp']).tolist()[0].tolist()) + 
#                       ((temp.loc[temp['date'] == row['date']])['humidity']).tolist()[0].tolist() +  
#                       (tmp.loc[tmp.index[0]]).tolist()
                      

(564, 16)


100%|████████████████████████████████████████████████████████████████████████████████| 564/564 [02:49<00:00,  3.32it/s]


In [21]:
result = pd.DataFrame(result, columns=['address_uuid', 'volume'])

In [23]:
result.to_csv(os.path.join('..', 'model', 'volume.csv'))
result.to_csv(os.path.join('mnt', 'result', 'volume.csv'))


OSError: Cannot save file into a non-existent directory: 'mnt\result'